In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

24/06/01 16:25:46 WARN Utils: Your hostname, Arbnors-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.10.0.115 instead (on interface en0)
24/06/01 16:25:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/01 16:25:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/01 16:25:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark = SparkSession.builder \
    .appName("Working with Datatypes and Joins") \
    .config("spark.driver.host", "driver-hostname") \
    .config("spark.driver.port", "driver-port") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .getOrCreate()

24/06/01 16:25:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df = spark.read.format("csv")\
	.option("header", "true")\
    .option("inferSchema", "true")\
    .load("data/retail-data/by-day/2010-12-01.csv")

df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [4]:
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [5]:
dfFiltered = df.filter("UnitPrice > 5")
dfFiltered.selectExpr("UnitPrice").show(10)

+---------+
|UnitPrice|
+---------+
|     7.65|
|     9.95|
|     5.95|
|     5.95|
|     7.95|
|     7.95|
|     5.95|
|     18.0|
|     6.95|
|     7.65|
+---------+
only showing top 10 rows



In [6]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
df.createOrReplaceTempView("dfTable")

In [8]:
from pyspark.sql.functions import lit

In [9]:
df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

In [10]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [11]:
from pyspark.sql.functions import col

In [12]:
df.where(col("InvoiceNo") != 536365).select("InvoiceNo","Description").show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [13]:
df.where(col("UnitPrice") > 5).select("InvoiceNo","Description", "UnitPrice").show(10, False)

+---------+---------------------------------+---------+
|InvoiceNo|Description                      |UnitPrice|
+---------+---------------------------------+---------+
|536365   |SET 7 BABUSHKA NESTING BOXES     |7.65     |
|536367   |BOX OF VINTAGE ALPHABET BLOCKS   |9.95     |
|536367   |HOME BUILDING BLOCK WORD         |5.95     |
|536367   |LOVE BUILDING BLOCK WORD         |5.95     |
|536367   |RECIPE BOX WITH METAL HEART      |7.95     |
|536367   |DOORMAT NEW ENGLAND              |7.95     |
|536369   |BATH BUILDING BLOCK WORD         |5.95     |
|536370   |POSTAGE                          |18.0     |
|536373   |WOOD S/3 CABINET ANT WHITE FINISH|6.95     |
|536373   |SET 7 BABUSHKA NESTING BOXES     |7.65     |
+---------+---------------------------------+---------+
only showing top 10 rows



In [14]:
from pyspark.sql.functions import instr

In [15]:
priceFilter = col("UnitPrice") > 600 

In [16]:
descFilter = instr(df.Description, "POSTAGE") >= 1

In [17]:
df.where(df.StockCode.isin("DOT")).where( priceFilter | descFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      NULL|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      NULL|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



let’s imagine that we found out that we mis-recorded the quantity in our retail dataset 
and the true quantity is equal to (the current quantity * the unit price)2 + 5

In [18]:
from pyspark.sql.functions import expr, pow

In [19]:
trueQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerID"), trueQuantity.alias("Quantity Updated")).show(5)

+----------+------------------+
|CustomerID|  Quantity Updated|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 5 rows



In [20]:
#Test
df.withColumn("CustomerID",expr(" CustomerID ||','|| Country")).show(10, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------------------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID            |Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------------------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0,United Kingdom|United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0,United Kingdom|United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0,United Kingdom|United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0,United Kingdom|United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.    

In [21]:
from pyspark.sql.functions import round

In [22]:
df.select(
    col("InvoiceNo"),
    round(col("UnitPrice").cast("int")).alias("RoundedUnitPrice"),
).show(10)

+---------+----------------+
|InvoiceNo|RoundedUnitPrice|
+---------+----------------+
|   536365|               2|
|   536365|               3|
|   536365|               2|
|   536365|               3|
|   536365|               3|
|   536365|               7|
|   536365|               4|
|   536366|               1|
|   536366|               1|
|   536367|               1|
+---------+----------------+
only showing top 10 rows



In [23]:
from pyspark.sql.functions import initcap, lower, upper, lit, ltrim, rtrim, rpad, lpad, trim, length

In [24]:
df.select(initcap(col("Description"))).show(5)

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
+--------------------+
only showing top 5 rows



In [25]:
df.select(col("Description"), lower(col("Description")), upper(col("Description"))).show(10, False)

+-----------------------------------+-----------------------------------+-----------------------------------+
|Description                        |lower(Description)                 |upper(Description)                 |
+-----------------------------------+-----------------------------------+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |white hanging heart t-light holder |WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |white metal lantern                |WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |cream cupid hearts coat hanger     |CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|knitted union flag hot water bottle|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |red woolly hottie white heart.     |RED WOOLLY HOTTIE WHITE HEART.     |
|SET 7 BABUSHKA NESTING BOXES       |set 7 babushka nesting boxes       |SET 7 BABUSHKA NESTING BOXES       |
|GLASS STA

In [26]:
df.select(ltrim(lit("  HELLO  ")).alias("ltrim"),rtrim(lit("  HELLO  ")).alias("rtrim"),\
          trim(lit("  HELLO  ")).alias("trim"),\
          lpad(lit("HELLO"), 3, " ").alias("lp"),\
          rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)

+-------+-------+-----+---+----------+
|  ltrim|  rtrim| trim| lp|        rp|
+-------+-------+-----+---+----------+
|HELLO  |  HELLO|HELLO|HEL|HELLO     |
|HELLO  |  HELLO|HELLO|HEL|HELLO     |
+-------+-------+-----+---+----------+
only showing top 2 rows



In [27]:
df.where(length(col("Country")) < 8).show(10)

+---------+---------+--------------------+--------+-------------------+---------+----------+-------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+-------+
|   536370|    22728|ALARM CLOCK BAKEL...|      24|2010-12-01 08:45:00|     3.75|   12583.0| France|
|   536370|    22727|ALARM CLOCK BAKEL...|      24|2010-12-01 08:45:00|     3.75|   12583.0| France|
|   536370|    22726|ALARM CLOCK BAKEL...|      12|2010-12-01 08:45:00|     3.75|   12583.0| France|
|   536370|    21724|PANDA AND BUNNIES...|      12|2010-12-01 08:45:00|     0.85|   12583.0| France|
|   536370|    21883|    STARS GIFT TAPE |      24|2010-12-01 08:45:00|     0.65|   12583.0| France|
|   536370|    10002|INFLATABLE POLITI...|      48|2010-12-01 08:45:00|     0.85|   12583.0| France|
|   536370|    21791|VINTAGE HEADS AND...|      24|2010-12-01 08:45:00|     1.25|   12583.0

In [28]:
from pyspark.sql.functions import current_date, current_timestamp

In [29]:
dateDf = spark.range(10).withColumn("Current date", current_date()).withColumn("Current TimeStamp", current_timestamp())

In [30]:
dateDf.createOrReplaceTempView("dateTable")

In [31]:
dateDf.printSchema()

root
 |-- id: long (nullable = false)
 |-- Current date: date (nullable = false)
 |-- Current TimeStamp: timestamp (nullable = false)



In [32]:
dateDf.show(10, False)

+---+------------+-----------------------+
|id |Current date|Current TimeStamp      |
+---+------------+-----------------------+
|0  |2024-06-01  |2024-06-01 16:26:00.657|
|1  |2024-06-01  |2024-06-01 16:26:00.657|
|2  |2024-06-01  |2024-06-01 16:26:00.657|
|3  |2024-06-01  |2024-06-01 16:26:00.657|
|4  |2024-06-01  |2024-06-01 16:26:00.657|
|5  |2024-06-01  |2024-06-01 16:26:00.657|
|6  |2024-06-01  |2024-06-01 16:26:00.657|
|7  |2024-06-01  |2024-06-01 16:26:00.657|
|8  |2024-06-01  |2024-06-01 16:26:00.657|
|9  |2024-06-01  |2024-06-01 16:26:00.657|
+---+------------+-----------------------+



In [33]:
from pyspark.sql.functions import date_add, date_sub, datediff, months_between, to_date

In [34]:
dateDf.select(datediff(col("Current Date"), lit("2022-05-30"))).show(1)

+----------------------------------+
|datediff(Current Date, 2022-05-30)|
+----------------------------------+
|                               733|
+----------------------------------+
only showing top 1 row



In [35]:
dateDf.select(date_sub(col("Current date"), 5), date_add(col("Current date"), 5)).show(1)

+-------------------------+-------------------------+
|date_sub(Current date, 5)|date_add(Current date, 5)|
+-------------------------+-------------------------+
|               2024-05-27|               2024-06-06|
+-------------------------+-------------------------+
only showing top 1 row



In [36]:
dateDf.withColumn("week_ago", date_sub(col("Current date"), 7))\
	.select(datediff(col("week_ago"), col("Current date"))).show(1)


dateDf.select(
      to_date(lit("2016-01-01")).alias("start"),
      to_date(lit("2017-05-22")).alias("end"))\
      .select(months_between(col("start"), col("end"))).show(1)

+--------------------------------+
|datediff(week_ago, Current date)|
+--------------------------------+
|                              -7|
+--------------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



In [37]:

dateDf.select(
      to_date(lit("2016-01-01")).alias("start"),
      to_date(lit("2017-05-22")).alias("end"))\
      .select(months_between(col("start"), col("end"))).show(1)

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



In [38]:
from pyspark.sql.functions import struct

complexDf = df.select(struct("Description", "InvoiceNo").alias("complex"))

complexDf.createOrReplaceTempView("complexDf")

In [39]:
complexDf.show(10, False)

+---------------------------------------------+
|complex                                      |
+---------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER, 536365} |
|{WHITE METAL LANTERN, 536365}                |
|{CREAM CUPID HEARTS COAT HANGER, 536365}     |
|{KNITTED UNION FLAG HOT WATER BOTTLE, 536365}|
|{RED WOOLLY HOTTIE WHITE HEART., 536365}     |
|{SET 7 BABUSHKA NESTING BOXES, 536365}       |
|{GLASS STAR FROSTED T-LIGHT HOLDER, 536365}  |
|{HAND WARMER UNION JACK, 536366}             |
|{HAND WARMER RED POLKA DOT, 536366}          |
|{ASSORTED COLOUR BIRD ORNAMENT, 536367}      |
+---------------------------------------------+
only showing top 10 rows



In [40]:
complexDf.select("complex.Description").show(5, False)

+-----------------------------------+
|Description                        |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+
only showing top 5 rows



In [41]:
complexDf.select("complex.InvoiceNo").show(5, False)

+---------+
|InvoiceNo|
+---------+
|536365   |
|536365   |
|536365   |
|536365   |
|536365   |
+---------+
only showing top 5 rows



In [42]:
complexDf.select("complex.*").show(5, False)

+-----------------------------------+---------+
|Description                        |InvoiceNo|
+-----------------------------------+---------+
|WHITE HANGING HEART T-LIGHT HOLDER |536365   |
|WHITE METAL LANTERN                |536365   |
|CREAM CUPID HEARTS COAT HANGER     |536365   |
|KNITTED UNION FLAG HOT WATER BOTTLE|536365   |
|RED WOOLLY HOTTIE WHITE HEART.     |536365   |
+-----------------------------------+---------+
only showing top 5 rows



In [43]:
complexDf.select(col("complex").getField("Description")).show(5, False)

+-----------------------------------+
|complex.Description                |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+
only showing top 5 rows



In [44]:
from pyspark.sql.functions import split

df.select(split("Description", " ").alias("splitted")).show(2)

+--------------------+
|            splitted|
+--------------------+
|[WHITE, HANGING, ...|
|[WHITE, METAL, LA...|
+--------------------+
only showing top 2 rows



In [45]:
df.select(split("Description", " ").alias("splitted")).selectExpr("splitted[1]").show(10, False)

+-----------+
|splitted[1]|
+-----------+
|HANGING    |
|METAL      |
|CUPID      |
|UNION      |
|WOOLLY     |
|7          |
|STAR       |
|WARMER     |
|WARMER     |
|COLOUR     |
+-----------+
only showing top 10 rows



In [46]:
from pyspark.sql.functions import split, size, array_contains, count

df.select(size(split("Description", " ")).alias("splitted")).show(5, False)

+--------+
|splitted|
+--------+
|5       |
|3       |
|5       |
|6       |
|5       |
+--------+
only showing top 5 rows



In [47]:
df.select(array_contains(split("Description", " "), "METAL")).show(2, False)

+------------------------------------------------+
|array_contains(split(Description,  , -1), METAL)|
+------------------------------------------------+
|false                                           |
|true                                            |
+------------------------------------------------+
only showing top 2 rows



In [48]:
df.select(count(array_contains(split("Description", " "), "METAL"))).show(2, False)

+-------------------------------------------------------+
|count(array_contains(split(Description,  , -1), METAL))|
+-------------------------------------------------------+
|3098                                                   |
+-------------------------------------------------------+



In [49]:
from pyspark.sql.functions import count_if

df.select(count_if(array_contains(split("Description", " "), "METAL") == "true")).show(5)

+-------------------------------------------------------------------+
|count_if((array_contains(split(Description,  , -1), METAL) = true))|
+-------------------------------------------------------------------+
|                                                                105|
+-------------------------------------------------------------------+



In [50]:
#Expload function returns a new row for each element in the given array or map. EX: from [WHITE, HANGING, HEART, T-LIGHT, HOLDER] 
# it creates four new rows with all the other data.
#|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|WHITE      |
#|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HANGING    |
#|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HEART      |
#|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|T-LIGHT    |




from pyspark.sql.functions import explode

df.withColumn("dataSplitted", split("Description", " "))\
    .withColumn("explodeData", explode("dataSplitted"))\
    .select("Description", "dataSplitted", "explodeData").show(10, False)

+----------------------------------+----------------------------------------+-----------+
|Description                       |dataSplitted                            |explodeData|
+----------------------------------+----------------------------------------+-----------+
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|WHITE      |
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HANGING    |
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HEART      |
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|T-LIGHT    |
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HOLDER     |
|WHITE METAL LANTERN               |[WHITE, METAL, LANTERN]                 |WHITE      |
|WHITE METAL LANTERN               |[WHITE, METAL, LANTERN]                 |METAL      |
|WHITE METAL LANTERN               |[WHITE, METAL, LANTERN]                 |LANTERN    |
|CREAM CUP

# JSON

In [51]:
jsonDF = spark.range(1).selectExpr("""'{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")

In [52]:
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[2]").alias("column"),
              json_tuple(col("jsonString"), "myJSONKey")).show(1, False)

+------+-----------------------+
|column|c0                     |
+------+-----------------------+
|3     |{"myJSONValue":[1,2,3]}|
+------+-----------------------+



In [53]:
def power3(num):
    return num ** 3

In [54]:
power3(2.0)

8.0

In [55]:
udfExample = spark.range(10).toDF("nums")

In [56]:
udfExample.show(3)

+----+
|nums|
+----+
|   0|
|   1|
|   2|
+----+
only showing top 3 rows



In [57]:
from pyspark.sql.functions import udf

power3udf = udf(power3)
spark.udf.register("power3", power3udf)

In [58]:
udfExample.select(power3udf(col("nums"))).show(6)

+------------+
|power3(nums)|
+------------+
|           0|
|           1|
|           8|
|          27|
|          64|
|         125|
+------------+
only showing top 6 rows



In [59]:
udfExample.selectExpr("power3(nums)").show(6)

+------------+
|power3(nums)|
+------------+
|           0|
|           1|
|           8|
|          27|
|          64|
|         125|
+------------+
only showing top 6 rows



In [60]:
person = spark.createDataFrame([
  (0, "Bill Chambers", 0, [100]),
  (1, "Matei Zaharia", 1, [500, 250, 100]),
  (2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
  (0, "Masters", "School of Information", "UC Berkeley"),
  (1, "Masters1", "School of Information", "UC Berkeley"),
  (1, "Masters2", "School of Information", "UC Berkeley"),
  (2, "Masters3", "EECS", "UC Berkeley"),
  (1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
  (500, "Vice President"),
  (250, "PMC Member"),
  (100, "Contributor")])\
.toDF("id", "status")

In [61]:
person.show()
sparkStatus.show()
graduateProgram.show(10, False)

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+

+---+--------+---------------------+-----------+
|id |degree  |department           |school     |
+---+--------+---------------------+-----------+
|0  |Masters |School of Information|UC Berkeley|
|1  |Masters1|School of Information|UC Berkeley|
|1  |Masters2|School of Information|UC Berkeley|
|2  |Masters3|EECS                 |UC Berkeley|
|1  |Ph.D.   |EECS                 |UC Berkeley|
+---+--------+---------------------+-----------+



In [62]:
joinCond = person["graduate_program"] == graduateProgram["id"]

In [63]:
innerJoin = person.join(graduateProgram, joinCond)

In [64]:
innerJoin.show()

+---+----------------+----------------+---------------+---+--------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id|  degree|          department|     school|
+---+----------------+----------------+---------------+---+--------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0| Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|Masters1|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|Masters2|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|Masters1|School of Informa...|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|Masters2|School of Informa...|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D.|              

In [65]:
outerJoin = person.join(graduateProgram, joinCond, "outer")
outerJoin.show()

+----+----------------+----------------+---------------+---+--------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id|  degree|          department|     school|
+----+----------------+----------------+---------------+---+--------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0| Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|Masters1|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|Masters2|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|Masters1|School of Informa...|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|Masters2|School of Informa...|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D.|    

In [66]:
left_outer = person.join(graduateProgram, joinCond, "left_outer")
left_outer.show()

+---+----------------+----------------+---------------+---+--------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id|  degree|          department|     school|
+---+----------------+----------------+---------------+---+--------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0| Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D.|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|Masters2|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|Masters1|School of Informa...|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|Masters2|School of Informa...|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|Masters1|School of Info

In [67]:
right_outer = person.join(graduateProgram, joinCond, "right_outer")
right_outer.show()

+----+----------------+----------------+---------------+---+--------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id|  degree|          department|     school|
+----+----------------+----------------+---------------+---+--------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0| Masters|School of Informa...|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|Masters1|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|Masters1|School of Informa...|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|Masters2|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|Masters2|School of Informa...|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D.|                EECS|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D.|    

In [68]:
#Left anti joins are the opposite of left semi joins. Like left semi joins, they do not actually include any values from the right DataFrame. They only compare values to see if the value exists in the second DataFrame. However, rather than keeping the values that exist in the second DataFrame, 
#they keep only the values that do not have a corresponding key in the second DataFrame
left_semi = graduateProgram.join(person, joinCond, "left_semi")
left_semi.show()

+---+--------+--------------------+-----------+
| id|  degree|          department|     school|
+---+--------+--------------------+-----------+
|  0| Masters|School of Informa...|UC Berkeley|
|  1|Masters1|School of Informa...|UC Berkeley|
|  1|Masters2|School of Informa...|UC Berkeley|
|  1|   Ph.D.|                EECS|UC Berkeley|
+---+--------+--------------------+-----------+



In [69]:
# Opposite of left_join
left_semi = graduateProgram.join(person, joinCond, "left_anti")
left_semi.show()

+---+--------+----------+-----------+
| id|  degree|department|     school|
+---+--------+----------+-----------+
|  2|Masters3|      EECS|UC Berkeley|
+---+--------+----------+-----------+



In [70]:
cross_join = graduateProgram.join(person, joinCond, "cross")
cross_join.show(10,False)

+---+--------+---------------------+-----------+---+----------------+----------------+---------------+
|id |degree  |department           |school     |id |name            |graduate_program|spark_status   |
+---+--------+---------------------+-----------+---+----------------+----------------+---------------+
|0  |Masters |School of Information|UC Berkeley|0  |Bill Chambers   |0               |[100]          |
|1  |Masters1|School of Information|UC Berkeley|1  |Matei Zaharia   |1               |[500, 250, 100]|
|1  |Masters1|School of Information|UC Berkeley|2  |Michael Armbrust|1               |[250, 100]     |
|1  |Masters2|School of Information|UC Berkeley|1  |Matei Zaharia   |1               |[500, 250, 100]|
|1  |Masters2|School of Information|UC Berkeley|2  |Michael Armbrust|1               |[250, 100]     |
|1  |Ph.D.   |EECS                 |UC Berkeley|1  |Matei Zaharia   |1               |[500, 250, 100]|
|1  |Ph.D.   |EECS                 |UC Berkeley|2  |Michael Armbrust|1   